In [1]:
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import sqlite3
from time import time, localtime, sleep, strftime, mktime
from datetime import datetime, timedelta

session = Session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
         'Referer': 'https://www.yna.co.kr/'}
searchDate = '20221201'
nday = 365

In [4]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [5]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [39]:
url = 'https://www.yna.co.kr/search/index'
urls = list()
urls.append((url, 0)) # 깊이 제한
seens = list()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

robots = dict()

categories = {'정치': '01', '경제': '02', '사회': '05', '금융증권산업': '03%2C04', '사건사고': '06', '문화': '07', '생활건강': '08', 'IT과학' : '09', '북한': '10', '국제': '11', '스포츠': '12', '연예': '0712'}

url_exp = 'https://www.yna.co.kr/search/index?query=%EC%82%AC%ED%9A%8C'

In [56]:
today = datetime.today()
uptodate = datetime.strptime(searchDate, '%Y%m%d')

# aday = 60*60*24

# days_ago = today - timedelta(days=1)
# endday = days_ago.strftime('%Y%m%d')
# print(endday)

for t in range(nday):
    days_ago = today - timedelta(days=t)
    theday = days_ago.strftime('%Y%m%d')
    break


for cate, code in categories.items():
    params = {
    'query': cate,
    'ctype': 'A',
    'from': theday,
    'to': theday,
    'period': 'diy',
    'div_code': code
}

20221204


In [2]:
js ='https://rd.yna.co.kr/global/js/ns.js?v=120221129_0955'
cb = 'https://ars.yna.co.kr/api/v2/search.asis?query=%EB%8B%A4.&ctype=A&div_code=09&page_size=10&channel=basic_kr'

In [3]:
# resp = get(url, params=params, headers=headers) # 깊이 제한
resp = get(cb, headers=headers) # 깊이 제한

# resp.status_code, resp.headers 
# resp.text[:1000]
dom = BeautifulSoup(resp.text, 'html.parser')
# dom.select('.title')
dom
# hList = dom.select('#article_list.cts_atclst .tt2')

## 크롬 DevTools에서 Elements 탭에서 보이는 HTML과 Network 탭에서 response.text의 결과가 다릅니다.

{"CONVERT_TIME":0,"ctype":"A","LOAD":113,"_V":"NEW","channel":"basic_kr","KR_ARTICLE":{"result":[{"CONTENTS_ID":"AKR20221205120300009","URGENCY":"3","DIST_TIME":"1729","WRITER_NAME":"임화섭","TITLE":"세계 최고성능 전파망원경, 구상 30년만에 남아공·호주서 착공","DIST_DATE":"20221205","INNER_FILE_INFO":"\/etc\/inner\/KR\/2022\/12\/05\/AKR20221205120300009_04_i_T.jpg","TEXT_BODY":"..., 블랙홀 주변의 환경, 중력파의 전파(傳播) 등 <b>다&lt;\/b&gt;양한 천문학 연구가 가능해질 것으로 전망된다.\n  ... 197개가 설치된다. 안테나간 최대 거리는 150㎞<b>다&lt;\/b&gt;.\n    SKA-미드는 현존 최고 성능 시설인 미국 ...0여개 기관들이 참여했다.\n    SKAO 사무총장인 필 <b>다&lt;\/b&gt;이아몬드 맨체스터대 물리천문학부 교수는 이번 ..."},{"CONTENTS_ID":"AKR20221205113200009","URGENCY":"3","DIST_TIME":"1703","WRITER_NAME":"이주영","TITLE":""박쥐, 데스메탈 가수처럼 7옥타브 넘나드는 소리 낸다"","DIST_DATE":"20221205","INNER_FILE_INFO":"\/etc\/inner\/KR\/2022\/12\/05\/AKR20221205113200009_01_i_T.jpg","TEXT_BODY":"... 같은 기법을 사용한다는 것이다.\n    연구팀은 <b>다&lt;\/b&gt;우벤토 박쥐 5마리에서 채취한 후두를 이용한 ...부터 120㎑의 고주파까지 낼 수 있다. 저주파로는 <b>다&lt;\/b&gt;른 박쥐들과 소통하고 고주파는 비행경로 탐색, ... 희생이 따른다며 "

In [ ]:
# crawl to db
con = sqlite3.connect('news_Crawl')
cur = con.cursor()

In [ ]:
cur.execute('''
	INSERT INTO news
	(URL, title, content, dateNews, keywords, category, dataInput)
	VALUES (:url, :title, :content, :dateNews, :keywords, :category, :dataInput);
''', {'url': url, 'title': hList, 'content': news, 'dateNews': dateNews, 'keywords': keywords, 'category': cate})

In [ ]:
while urls:
    # URLs Pool
    seed = urls.pop(-1) # Queue=BFS, Stack=DFS
    seens.append(seed[0]) # 깊이 제한
    
    # Robots.txt
#     if seed not in robots:
#         robots[seed] = robotParser(seed)
#     rp = robots[seed]
#     print('[Robots.txt]', seed, canFetch(rp, seed)) # True일때만, 지금은무시
    
    # Focused Crawling
#     if seed[1] > 2: # 자의적 => 휴리스틱
#         continue
        
    resp = get(seed[0], headers=headers) # 깊이 제한
    if resp.status_code == 200 and 'Content-Type'.lower() in resp.headers:
                                   # content-type이 있을때만
        if 'text/html' in\
           [_.strip() for _ in resp.headers['Content-Type'].split(';')]: #바꿔야함
            dom = BeautifulSoup(resp.text, 'html.parser')
            domlist = dom.select('#article_list.search_news_list.dis_none')
            aList = dom.select('#article_list.cts_atclst a')
            hList = dom.select('#article_list.cts_atclst .tt2')
            news = dom.select_one('.story-news\ article p')
            if len(aList) > 0:
                for el in aList[1:6]:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)

                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            if len(hList) > 0:
                for el in hList:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)
# https://n.news.naver.com/mnews/article/119/0002640090?sid=102
                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            news = dom.select_one('#ct #contents')
            if news:
                fileName = re.search('(\d{10})[?]sid=(\d{3})$', resp.url)
                with open(
                    '/content/drive/MyDrive/dataset/paper/{}-{}.txt'.format(fileName.group(2),
                                              fileName.group(1)),
                    'w', encoding='utf8') as f:
                    f.write(news.text.strip())
                for img in news.select('img[data-src]'):
                    urls.append((urljoin(resp.url, img.attrs['data-src']), seed[1]+1))
#             for el in dom.select('[src], [href]'): # [:10] 너비제한
#                 url = el.attrs['src' if 'src' in el.attrs else 'href']
#                 nextUrl = urljoin(seed[0], url) # 깊이 제한
#                 urlParams = urlparse(nextUrl)

#                 # URL 체크
#                 if nextUrl not in seens and\
#                    nextUrl not in [_[0] for _ in urls] and\
#                    urlParams.netloc == 'blog.naver.com' and\
#                    nextUrl.startswith('http'):
#                     # javascript, #fragment 제외시켜야 함
#                     # Focused = 특정 도메인으로 제한
#                     urls.append((nextUrl, seed[1]+1)) # 깊이 제한
        else:
            #image/jpeg
            ext = re.search(r'image/(png|jpeg|bmp|gif)', resp.headers['Content-Type'])
            if ext:
#                 fileName = re.search('(\d{8}_\d{3})', resp.url)
                fileName = resp.url.split('/')[-1]
                fileName += '.'+ ext.group(1)
                with open('/content/drive/MyDrive/dataset/news/'+fileName, 'wb') as fp:
                    fp.write(resp.content)
    print(len(urls), len(seens))